### End to End Demo for Cleanlab Studio Python API

1) Initialize Studio object with your API key

In [1]:
from cleanlab_studio import Studio

api_key = "<insert_api_key>"
studio = Studio(api_key)

2. Load dataset as pandas dataframe

In [2]:
import pandas as pd

df = pd.read_csv("https://s.cleanlab.ai/amazon-text-demo.csv")
print(df.head())

   index                                        review_text        asin  stars
0      0  This magazine was great for the times but as w...  B00005N7P0    ⭐⭐⭐
1      1  We ordered this magazine for our grandson (the...  B00005N7P0  ⭐⭐⭐⭐⭐
2      2  I didn't receive a full year.  I only receive ...  B00005N7P0     ⭐⭐
3      3  I was hoping for more technical than what was ...  B00005N7P0     ⭐⭐
4      4  I only received one copy of the mag so I could...  B00005N7P0     ⭐⭐


3. Upload dataset to Studio

In [3]:
dataset_id = studio.upload_dataset(df, "Amazon Reviews")
dataset_id

Uploading dataset...: 100%|██████████|
Generating schema...: 100%|██████████|
Ingesting Dataset...: 100%|██████████|


'4c42e71325f74cc39db16abad13c6d56'

4. Create Project

In [4]:
project_id = studio.create_project(
    dataset_id,
    "Amazon Review Project",
    "text",
    label_column="stars",
    text_column="review_text",
    model_type="fast",
)
project_id

'c3b877bff2a94efdb94324ff811edf85'

5. Get cleanset ID for project

In [5]:
cleanset_id = studio.get_latest_cleanset_id(project_id)
cleanset_id

'd6f31ada433840b19b3b685c04a9afa3'

6. Poll for cleanset status until cleanset is ready

In [6]:
studio.poll_cleanset_status(cleanset_id)

Cleanset Progress: / Step 5/5, Ready for review! 


True

7. Download cleanlab columns

In [7]:
cleanlab_columns_df = studio.download_cleanlab_columns(cleanset_id)
cleanlab_columns_df.head()

,index,cleanlab_issue,cleanlab_label_quality,cleanlab_suggested_label,cleanlab_clean_label,cleanlab_outlier
0,0,False,0.580450,None,None,False
1,1,False,0.721680,None,None,False
2,2,True,0.084521,⭐⭐⭐,None,False
3,3,False,0.548215,None,None,False
4,4,True,0.348590,⭐⭐⭐,None,False
